# load pennai ML options

In [1]:
import json                                                                          
import numpy as np                                                                   
from json import JSONDecoder                                                         
from functools import partial                                                        
                                                                         
alldata= json.load(open('../../../dockers/dbmongo/files/projects.json'),encoding="utf-8")

# path = 'projects_newlines.json'                                
# data,alldata=[],[]                                                                   
# with open(path) as f:                                                                
#     for line in f:                                                                   
#         alldata.append(json.loads(line))                                             
#     # for data in json_parse(f):                                                     
#     #     print(data)                                                                
#     #     alldata.append(data)                                                       
pennai_ml_opts = {}                                                                         
import pdb                                                                           
from sklearn.model_selection import ParameterGrid                                    
                                                                                     
original_pennai_ml_opts = {}
for d in alldata:                                                                    
    print('ml:',d['name'])                                                           
    if (d['category'] != 'classification'
        or 'XGB' in d['name']):
        print('skipping')
        continue
                                                                                     
    params = d['schema'].keys()                                                      
    choices = {}                                                                     
    for p in params:                               
        if 'values' in d['schema'][p]['ui']:
            choices[p] = d['schema'][p]['ui']['values']                                 
        else:
            choices[p] = d['schema'][p]['ui']['choices']                                 
        print('param:',p,', options:',choices[p])
    # add None to DecisionTreeClassifier's max_depth option.
    if d['name'] == 'DecisionTreeClassifier':
        choices['max_depth'].append(None)
#     pdb.set_trace()
    original_pennai_ml_opts[d['name']] = choices.keys()
    param_grid = list(ParameterGrid(choices))                                        
#     print('param_grid contains',len(param_grid),'options')                           
    pennai_ml_opts[d['name']] = param_grid                                                  
print('pennai_ml_opts:',len(pennai_ml_opts))      
for key in pennai_ml_opts.keys():
    print(key,'options:',len(pennai_ml_opts[key]))

ml: BernoulliNB
param: alpha , options: [0.001, 0.01, 0.1, 1, 10, 100]
param: binarize , options: [0, 0.25, 0.5, 0.75, 1]
param: fit_prior , options: ['true', 'false']
ml: GaussianNB
ml: MultinomialNB
param: alpha , options: [0.001, 0.01, 0.1, 1.0, 10, 100]
param: fit_prior , options: ['true', 'false']
ml: DecisionTreeClassifier
param: criterion , options: ['gini', 'entropy']
param: max_depth , options: [3, 5, 10]
param: min_samples_split , options: [2, 5, 10, 20]
param: min_samples_leaf , options: [1, 5, 10, 20]
param: min_weight_fraction_leaf , options: [0.0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45]
param: max_features , options: ['sqrt', 'log2', 'None']
ml: ExtraTreesClassifier
param: n_estimators , options: [100, 500]
param: criterion , options: ['gini', 'entropy']
param: max_features , options: ['sqrt', 'log2', 'None']
param: min_samples_split , options: [2, 5, 10, 20]
param: min_samples_leaf , options: [1, 5, 10, 20]
param: bootstrap , options: ['true', 'false']
ml: Grad

## method to extend ML parameter options with default parameters if not present

In [2]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import BernoulliNB, GaussianNB, MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.linear_model import LogisticRegression
default_params = {}
default_params['DecisionTreeClassifier'] = DecisionTreeClassifier().get_params()
default_params['RandomForestClassifier'] = RandomForestClassifier().get_params()
default_params['ExtraTreesClassifier'] = ExtraTreesClassifier().get_params()
default_params['GradientBoostingClassifier'] = GradientBoostingClassifier().get_params()
default_params['BernoulliNB'] = BernoulliNB().get_params()
default_params['GaussianNB'] = GaussianNB().get_params()
default_params['MultinomialNB'] = MultinomialNB().get_params()
default_params['KNeighborsClassifier'] = KNeighborsClassifier().get_params()
default_params['LinearSVC'] = LinearSVC().get_params()
default_params['SVC'] = SVC().get_params()
default_params['LogisticRegression'] = LogisticRegression().get_params()

##
# format true, false and none variables from json to python
def json_to_py_params(params):
    for k,v in params.items():
        if type(v) is str:
            if v == 'true':
                params[k] = True
            elif v == 'false':
                params[k] = False
            elif v == 'None':
                params[k] == None
            else:
                params[k] = v.lower() 
    return params

def extend_params(ml,params):
    missing_params = [(k,v) for k,v in default_params[ml].items() if k not in params]
    for k,v in missing_params:
        params[k] = v
    return params



In [3]:
print('extending parameters...')
pennai_ml_opts_ext = {}
for k,v in pennai_ml_opts.items():
    pennai_ml_opts_ext[k] = []
    for params in v:
        pennai_ml_opts_ext[k].append(json_to_py_params(extend_params(k,params)))

extending parameters...


In [4]:
pennai_ml_opts['RandomForestClassifier']

[{'bootstrap': True,
  'criterion': 'gini',
  'max_features': 'sqrt',
  'min_samples_leaf': 1,
  'min_samples_split': 2,
  'min_weight_fraction_leaf': 0.0,
  'n_estimators': 100,
  'class_weight': None,
  'max_depth': None,
  'max_leaf_nodes': None,
  'min_impurity_decrease': 0.0,
  'min_impurity_split': None,
  'n_jobs': None,
  'oob_score': False,
  'random_state': None,
  'verbose': 0,
  'warm_start': False},
 {'bootstrap': True,
  'criterion': 'gini',
  'max_features': 'sqrt',
  'min_samples_leaf': 1,
  'min_samples_split': 2,
  'min_weight_fraction_leaf': 0.0,
  'n_estimators': 500,
  'class_weight': None,
  'max_depth': None,
  'max_leaf_nodes': None,
  'min_impurity_decrease': 0.0,
  'min_impurity_split': None,
  'n_jobs': None,
  'oob_score': False,
  'random_state': None,
  'verbose': 0,
  'warm_start': False},
 {'bootstrap': True,
  'criterion': 'gini',
  'max_features': 'sqrt',
  'min_samples_leaf': 1,
  'min_samples_split': 2,
  'min_weight_fraction_leaf': 0.05,
  'n_estima

# load pmlb results
filter out classifiers not in PennAI

In [5]:
import pandas as pd
import numpy as np

data = pd.read_csv('../../../mock_experiment/sklearn-benchmark5-data-edited.tsv.gz', sep='\t').fillna('')
print(data.head())
data['accuracy'] = data['accuracy'].apply(lambda x: round(x, 3))
print('loaded ',data['dataset'].unique().shape[0],'datasets and ', data['classifier'].unique().shape[0],'classifiers')
# subset data to classifiers used in PennAI
pennai_classifiers = ['LogisticRegression', 'RandomForestClassifier', 'SVC', 
                      'KNeighborsClassifier', 'DecisionTreeClassifier', 'GradientBoostingClassifier']
mask = np.array([c in pennai_classifiers for c in data['classifier'].values])
data = data.loc[mask,:]
print('datasets (',len(data['dataset'].unique()),')')
print('classifiers (',len(data['classifier'].unique()),'):',data['classifier'].unique())
for ml, df_g in data.groupby('classifier'):
    print('parameters for ',ml,'(',len(df_g['parameters'].unique()),'):',df_g['parameters'].unique()[:5])


   Unnamed: 0                                            dataset  \
0           0  GAMETES_Epistasis_2-Way_1000atts_0.4H_EDM-1_ED...   
1           1  GAMETES_Epistasis_2-Way_1000atts_0.4H_EDM-1_ED...   
2           2  GAMETES_Epistasis_2-Way_1000atts_0.4H_EDM-1_ED...   
3           3  GAMETES_Epistasis_2-Way_1000atts_0.4H_EDM-1_ED...   
4           4  GAMETES_Epistasis_2-Way_1000atts_0.4H_EDM-1_ED...   

           classifier                            parameters  accuracy  \
0  AdaBoostClassifier    learning_rate=0.01,n_estimators=10  0.479375   
1  AdaBoostClassifier   learning_rate=0.01,n_estimators=100  0.476875   
2  AdaBoostClassifier  learning_rate=0.01,n_estimators=1000  0.488125   
3  AdaBoostClassifier    learning_rate=0.01,n_estimators=50  0.483750   
4  AdaBoostClassifier   learning_rate=0.01,n_estimators=500  0.495625   

    macrof1  bal_accuracy  
0  0.470857      0.479375  
1  0.476497      0.476875  
2  0.488115      0.488125  
3  0.483652      0.483750  
4  0.495609 

In [6]:
data.groupby('classifier').count()

,Unnamed: 0,dataset,parameters,accuracy,macrof1,bal_accuracy
classifier,,,,,,
DecisionTreeClassifier,25410,25410,25410,25410,25410,25410
GradientBoostingClassifier,769900,769900,769900,769900,769900,769900
KNeighborsClassifier,8831,8831,8831,8831,8831,8831
LogisticRegression,39145,39145,39145,39145,39145,39145
RandomForestClassifier,126602,126602,126602,126602,126602,126602
SVC,143179,143179,143179,143179,143179,143179


# convert from sklearn-style parameter formatting to dictionary

In [7]:
def fix_params(p):
    fixed_params = {k:v for k,v in [tuple(ps.split('=')) for ps in filter(None, p.split(','))]}
    for k,v in fixed_params.items():
        try:
            fixed_params[k] = int(v)
        except ValueError:
            try:     
                fixed_params[k] = float(v)
            except ValueError:
                if fixed_params[k] == 'None':
                    fixed_params[k] = None 
                else:
                    fixed_params[k] = str(v).lower()
                pass
    return fixed_params
# datanew = data.copy()
data_formatted = data.copy()
print('applying fix_params')
data_formatted['parameters'] = data_formatted['parameters'].apply(lambda x: fix_params(x))

data_formatted.head()

applying fix_params


,Unnamed: 0,dataset,classifier,parameters,accuracy,macrof1,bal_accuracy
175,175,GAMETES_Epistasis_2-Way_1000atts_0.4H_EDM-1_ED...,DecisionTreeClassifier,"{'min_weight_fraction_leaf': 0.0, 'max_feature...",0.507,0.507488,0.507500
176,176,GAMETES_Epistasis_2-Way_1000atts_0.4H_EDM-1_ED...,DecisionTreeClassifier,"{'min_weight_fraction_leaf': 0.0, 'max_feature...",0.476,0.476040,0.476250
177,177,GAMETES_Epistasis_2-Way_1000atts_0.4H_EDM-1_ED...,DecisionTreeClassifier,"{'min_weight_fraction_leaf': 0.0, 'max_feature...",0.507,0.506832,0.506875
178,178,GAMETES_Epistasis_2-Way_1000atts_0.4H_EDM-1_ED...,DecisionTreeClassifier,"{'min_weight_fraction_leaf': 0.0, 'max_feature...",0.490,0.489993,0.490000
179,179,GAMETES_Epistasis_2-Way_1000atts_0.4H_EDM-1_ED...,DecisionTreeClassifier,"{'min_weight_fraction_leaf': 0.0, 'max_feature...",0.512,0.512497,0.512500


## add default parameters to parameter dictionaries

In [8]:
def df_extend_params(x):
#     print(x)
    x.parameters = json_to_py_params(extend_params(x.classifier,x.parameters))
    return x
data_formatted_ext = data_formatted.copy()
print('extending pmlb data parameters...')
data_formatted_ext = data_formatted_ext.apply(lambda x: df_extend_params(x),axis=1)

extending pmlb data parameters...


# filter out any parameter combinations that PennAI can't recommend

In [9]:
def valid_param_combo(ml, params):
    if params in pennai_ml_opts_ext[ml]:
        return True
    else:
        return False
# print('unfiltered:',len(data_formatted_ext),'results with',
#      len(data_formatted_ext.parameters.apply(str).unique()),'unique parameters')
data_formatted_ext_filtered = data_formatted_ext#.loc[data_formatted_ext.classifier=='SVC']
print('calculating mask...')
mask = []
from tqdm import tqdm
for _, row in tqdm(data_formatted_ext_filtered.iterrows()):
    mask.append(valid_param_combo(row['classifier'],row['parameters']) )
print('done with mask')
# mask = mask | data_formatted_ext_filtered.classifier=='LogisticRegression'
data_formatted_ext_filtered = data_formatted_ext_filtered.loc[mask]
print('filtered data has',len(data_formatted_ext_filtered),'results with',
     len(data_formatted_ext_filtered.parameters.apply(str).unique()),'unique parameters')
data_formatted_ext_filtered.groupby('classifier').count()
# data_filtered = data.loc[lambda x: valid_param_combo(i['classifier'],i['parameters']) for i in x]


1it [00:00,  6.92it/s]

calculating mask...


1113067it [03:58, 4676.35it/s]


done with mask
filtered data has 52355 results with 381 unique parameters


,Unnamed: 0,dataset,parameters,accuracy,macrof1,bal_accuracy
classifier,,,,,,
DecisionTreeClassifier,4620,4620,4620,4620,4620,4620
GradientBoostingClassifier,7544,7544,7544,7544,7544,7544
KNeighborsClassifier,1980,1980,1980,1980,1980,1980
LogisticRegression,1968,1968,1968,1968,1968,1968
RandomForestClassifier,9212,9212,9212,9212,9212,9212
SVC,27031,27031,27031,27031,27031,27031


In [10]:
print('pennai_ml_opts:')
print(pennai_ml_opts_ext['GradientBoostingClassifier'][0].keys(),len(pennai_ml_opts_ext['GradientBoostingClassifier'][0].keys()))
print('data opts:')
print(data_formatted_ext_filtered.loc[data_formatted_ext_filtered.classifier=='GradientBoostingClassifier'].parameters.values[0].keys())
len(data_formatted_ext_filtered.loc[data_formatted_ext_filtered.classifier=='GradientBoostingClassifier'].parameters.values[0].keys())

pennai_ml_opts:
dict_keys(['learning_rate', 'max_depth', 'max_features', 'min_samples_leaf', 'min_samples_split', 'n_estimators', 'subsample', 'criterion', 'init', 'loss', 'max_leaf_nodes', 'min_impurity_decrease', 'min_impurity_split', 'min_weight_fraction_leaf', 'n_iter_no_change', 'presort', 'random_state', 'tol', 'validation_fraction', 'verbose', 'warm_start']) 21
data opts:
dict_keys(['loss', 'learning_rate', 'n_estimators', 'max_depth', 'max_features', 'criterion', 'init', 'max_leaf_nodes', 'min_impurity_decrease', 'min_impurity_split', 'min_samples_leaf', 'min_samples_split', 'min_weight_fraction_leaf', 'n_iter_no_change', 'presort', 'random_state', 'subsample', 'tol', 'validation_fraction', 'verbose', 'warm_start'])


21

In [11]:
for clf, dfg in data_formatted_ext_filtered.groupby('classifier'):
    print(clf,'unique params combos:',len(dfg.parameters.apply(str).unique()))
alg_params = data_formatted_ext_filtered['classifier']+'|'+data_formatted_ext_filtered['parameters'].apply(str)
print(len(alg_params.unique()),'unique algorithm/parameter combinations')

DecisionTreeClassifier unique params combos: 28
GradientBoostingClassifier unique params combos: 48
KNeighborsClassifier unique params combos: 12
LogisticRegression unique params combos: 12
RandomForestClassifier unique params combos: 56
SVC unique params combos: 225
381 unique algorithm/parameter combinations


# re-prune parameters from knowledgebase that were added as defaults (not in PennAI)

In [12]:
def prune_params(x):
#     pdb.set_trace()
    ml = x.classifier
    params = x.parameters
    param_opts = original_pennai_ml_opts[ml]
    x.parameters = {k:v for k,v in params.items() if k in param_opts}
    return x 
data_formatted_ext_filtered_pruned = data_formatted_ext_filtered.copy()
data_formatted_ext_filtered_pruned = data_formatted_ext_filtered_pruned.apply(
    lambda x: prune_params(x),axis=1)
    


In [13]:
for clf, dfg in data_formatted_ext_filtered_pruned.groupby('classifier'):
    print(clf,'unique params combos:',len(dfg.parameters.apply(str).unique()))
alg_params = (data_formatted_ext_filtered_pruned['classifier']+'|'
              +data_formatted_ext_filtered_pruned['parameters'].apply(str))
print(len(alg_params.unique()),'unique algorithm/parameter combinations')

DecisionTreeClassifier unique params combos: 28
GradientBoostingClassifier unique params combos: 48
KNeighborsClassifier unique params combos: 12
LogisticRegression unique params combos: 12
RandomForestClassifier unique params combos: 56
SVC unique params combos: 225
381 unique algorithm/parameter combinations


# write modified data to file

In [15]:
dfxfp = data_formatted_ext_filtered_pruned
dfxfp.drop(columns = ['Unnamed: 0'], inplace=True)

In [16]:
dfxfp.groupby('classifier').count()

,dataset,parameters,accuracy,macrof1,bal_accuracy
classifier,,,,,
DecisionTreeClassifier,4620,4620,4620,4620,4620
GradientBoostingClassifier,7544,7544,7544,7544,7544
KNeighborsClassifier,1980,1980,1980,1980,1980
LogisticRegression,1968,1968,1968,1968,1968
RandomForestClassifier,9212,9212,9212,9212,9212
SVC,27031,27031,27031,27031,27031


In [17]:
print(len(dfxfp),'total results')
dfxfp.rename(columns={'classifier':'algorithm'},inplace=True)
dfxfp.to_csv('sklearn-benchmark-data-knowledgebase-r6.tsv.gz',
                                                compression='gzip',index=False,sep='\t')

52355 total results


In [18]:
# make a small knowledge base for testing
test_data = dfxfp.sample(frac=0.1)
test_data.to_csv('sklearn-benchmark5-data-knowledgebase-small.tsv.gz',
                                                compression='gzip',index=False,sep='\t')